In [1]:
import sys
import os
lib = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities\droplet_dataset'
lib2 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\utilities'
lib3 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\data_analysis'
lib4 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy'
lib5 = r'D:\Technion studies\Keren Laboratory\python_playground\classifying-response-to-immunotherapy\scripts'
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)

In [2]:
import sklearn
from droplet_dataset import *
from utilities import *
from matplotlib import pyplot
import pandas as pd
from os.path import join
from classifying_cell_types import *

# 10/12/2020
For our collaborators to investigate the markers

In [105]:


SAMPLES_PATH = r'D:\Technion studies\Keren Laboratory\python_playground\outputs\classifying_cell_types\5.12.20'
MARKERS_PATH = r'D:\Technion studies\Keren Laboratory\Data\tables\ImmuneCellsMarkersUpdated_12.11.20.xlsx'
# OUTPUT_PATH = r'D:\Technion studies\Keren Laboratory\python_playground\output files'

samples = [subfolder for subfolder in os.listdir(SAMPLES_PATH) if not ('csv' in subfolder or 'xlsx' in subfolder)]
CSV_NAME = 'informative_conflict_table.csv'




In [106]:
def builds_cell_type_markers_table(df):
    """
    Extract cell-types markers from Excel file. the function can be used both with positive and negative markers df.
    :return: Dictionary containing for each cell-type (keys) the corresponding markers (values).
    """

    # Reorganize the table, the column names actually accouns the first row of the DataFrame
    columns = [v if not 'Unnamed' in v else np.NAN for v in list(df.columns)]
    df.loc[len(df)] = columns
    df = df.reindex([len(df)-1] + list(range(0, len(df)-1))).reset_index(drop=True)

    # convert df to dictionary markers table
    rm_nan = lambda lst: list([vv for vv in lst if not vv is np.NAN])
    df_rows = df.values.tolist()
    markers_table = {v[0]: rm_nan(v[1:]) for v in df_rows}
    markers_table = {k:flatten_list([ii.split(';') for ii in v]) for k,v in markers_table.items()}
    return markers_table

# Chek for overlap between group A and group B
Enough cell to have one cell-type from group A and one from B in order to be shown in the output table.
* PLEASE specify the groups

In [123]:
group_a = ['T cells']
group_b = ['Monocyte_immature']
#all celltypes group_b = ['T cells', 'CD4 helper T cells', 'CD8 Cytotoxic T cells', 'Regulatory T cells', 'Regulatory CD4 T cells', 'Regulatory CD8 T cells', 'Regulatory CD4_CD8 T cells', 'NKT cells', 'NK cells', 'B cells', 'Activated T cells', 'Senescence T cells', 'Terminal effector', 'Exhausted T cells', 'Stem_like T cells', 'Memory T cells', 'Memory CD4 T cells', 'Memory CD8 T cells', 'Memory CD4_CD8 T cells', 'Macrophage_immature', 'Macrophage_mature', 'Monocyte_immature', 'Monocyte_mature', 'cDCs_dendritic_cells', 'pDCs', 'myeloid cells_general_immature', 'myeloid cells_general_mature', 'Neutrophils', 'Granolocytes', 'Immune_general']




xls = pd.ExcelFile(MARKERS_PATH)
markers_table = builds_cell_type_markers_table(pd.read_excel(xls, 'and_or'))

markers = list(set(flatten_list([markers_table[c_type] for c_type in group_a])))
markers += list(set(flatten_list([markers_table[c_type] for c_type in group_b])))
markers

['CD3E', 'FCGR1A', 'CD14']

In [124]:
summary_df = pd.DataFrame(columns=['location']+markers)


for sample_id in samples:
    print(sample_id)
    data_path = join(SAMPLES_PATH, sample_id, f'{sample_id}.pkl')
    rna_sample = extract_droplet_data_from_pickle(data_path)
    
    c_inf = rna_sample.cells_information
    cells_indexes = [idx for idx, c in enumerate(c_inf) if is_there_overlap_in_lists(c.cell_type_list, group_a) 
                     and is_there_overlap_in_lists(c.cell_type_list, group_b)]
    
    for c_idx in cells_indexes:
        new_row = [f'sample: {sample_id} cell: {c_idx}']
        for marker in markers:
            mk_pl = [idx for idx, g in enumerate(rna_sample.gene_names) if g==marker]
            if mk_pl:
                new_row.append(rna_sample.counts[c_idx, mk_pl[0]]) 
            else:
                new_row.append(0)
        summary_df = summary_df.append(pd.DataFrame([new_row], columns=summary_df.columns))
    
    
summary_df


M100
M101
M102
M103
M104
M105
M106
M107
M108
M109
M110
M111
M112
M114
M115
M116
M118
M120
M121
M122
M123
M124
M125
M126
M127
M128
M129
M130
M131
M132
M133
M134
M135
M136
M137
M138
M139
M140
M141
M143
M144
M145
M146
M97
M98
M99


,location,CD3E,FCGR1A,CD14
0,sample: M111 cell: 798,8,1,1
0,sample: M111 cell: 2314,2,1,3
0,sample: M111 cell: 2809,1,1,6
0,sample: M111 cell: 3350,1,1,4
0,sample: M111 cell: 3489,8,4,3
0,sample: M111 cell: 3692,9,1,1
0,sample: M111 cell: 3699,1,1,4
0,sample: M111 cell: 4707,1,1,7
0,sample: M111 cell: 6555,1,1,5
0,sample: M123 cell: 2505,1,1,3


In [125]:
OUTPUT_PATH = fr'D:\Technion studies\Keren Laboratory\python_playground\outputs\building_informative_tables'
FILE_NAME = r'T_cells_Monocyte_immature.xlsx'


OUTPUT_PATH = join(OUTPUT_PATH, FILE_NAME)
summary_df.to_excel(OUTPUT_PATH, index=False)